In [9]:
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from collections import OrderedDict

In [10]:
data_path = Path('../data')

!ls $data_path

raw			       stability_embeddings_train.pkl
stability.csv		       stability_embeddings_val.pkl
stability_diversity_full.csv   stability.h5
stability_diversity_test.csv   stability_test.csv
stability_diversity_train.csv  stability_test.h5
stability_embeddings.pkl       stability_train.csv
stability_embeddings_test.pkl  stability_train.h5


In [11]:
def open_sets(base_path):
    sets = {}

    for path in base_path.glob('stability_*.csv'):
        fname = path.stem
        kind = fname.split('_')[1]

        df = pd.read_csv(path)
        sets[kind] = df
        
    return sets

sets = open_sets(data_path)
sets['train']

,sequence,consensus_stability_score
0,GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT,0.37
1,TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK,0.62
2,TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK,-0.03
3,GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT,1.41
4,TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT,1.11
...,...,...
7705,GSSKTQYEYDTKEEHQKAYEKFKKQGIPVTITQKNGKWFVQVE,0.80
7706,TIDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKPY,0.82
7707,TQDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKEY,0.66
7708,TTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTVWVTR,1.05


## Data Duplication

In [12]:
len(sets['train']['sequence'].unique()) != len(sets['train'])

False

## Alignments

In [13]:
from Bio import pairwise2

alignment = pairwise2.align.globalxx(sets['train']['sequence'][0], sets['train']['sequence'][1])
print(pairwise2.format_alignment(*alignment[0]))

GSSQETIEVE-DEEEARRVAKEL--R-KKGYEVK----D------E--R---RGNKWHVHRT------
     |     |  || |   ||  | ||  |      |      |  |   |   | | |       
-----T----LD--EA-R---ELVERAKK--E--GTGMDVNGQRFEDWREAER---W-V-R-EQEKNK
  Score=18



## Score per metrics

* More unique values => more stability?
* Does sequence length matter?


In [14]:
def append_to_key(d, k, v):
    if k in d:
        values = d[k]
        values.append(v)
        d[k] = values
        
    else:
        d[k] = [v]
        
def compute_mean_scores(d):
    new_d = dict()
    for k in d:
        mean_score = np.mean(d[k])
        new_d[k] = (mean_score, len(d[k]))
        
    return  OrderedDict(sorted(new_d.items()))

In [15]:
score_per_length = dict()
score_per_uniqueness = dict()

for idx, (seq, score) in sets['train'].iterrows():
    # score_per_length
    size = len(seq)
    append_to_key(score_per_length, size, score)
    
    
    # score_per_uniqueness => score per number of unique aminoacids in each seq
    uniques = set(seq)
    unique_size = len(uniques)
    
    append_to_key(score_per_uniqueness, unique_size, score)
    
    # count unique aminoacids
    for amino in uniques:
        uniques.add(amino)


In [16]:
compute_mean_scores(score_per_length)

OrderedDict([(43, (1.0939332224993072, 7218)),
             (46, (0.5659959349593496, 492))])

In [17]:
compute_mean_scores(score_per_uniqueness)

OrderedDict([(11, (1.1877272727272727, 22)),
             (12, (1.1719895287958115, 382)),
             (13, (1.1024068598462446, 1691)),
             (14, (1.04685173089484, 1531)),
             (15, (1.1661388550548113, 1642)),
             (16, (1.1496199999999999, 1000)),
             (17, (0.7856092436974791, 476)),
             (18, (0.8948214285714284, 168)),
             (19, (0.8083333333333332, 798))])

## By Score

In [18]:
sorted_train = sets['train'].sort_values(by='consensus_stability_score').reset_index()

In [19]:
alignment = pairwise2.align.globalxx(sorted_train['sequence'][0], sorted_train['sequence'][1])
print(pairwise2.format_alignment(*alignment[0]))

sorted_train.head(5)

RKWEE--IA-----ER----LREEFNIN-PEE-AREAVEKAGGNEEEAR--R-P---VK-------K--RL
       |      |     |        | | |  |  ||      |   | |   |        |  | 
-----TTI-KVNGQE-YTVPL-------SP-EQA--A--KA------A-KKRWPDYEV-QIHGNTYKVTR-
  Score=15



,index,sequence,consensus_stability_score
0,5537,RKWEEIAERLREEFNINPEEAREAVEKAGGNEEEARRPVKKRL,-0.26
1,122,TTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTYKVTR,-0.13
2,7529,TELKKKREEALKKGEEVRVKFNGIEIRITSEDAARKAVELLEK,-0.12
3,2520,GMADEEKLPPGWESRMSRSSGRVYYFNHITNASQWERPSGGSS,-0.11
4,2471,GSSGSLSDEDFKAVFGMTRSAFANLPLWKQQNPKKEKGLFGSS,-0.10


In [20]:
alignment = pairwise2.align.globalxx(
    sorted_train['sequence'][len(sorted_train)-1], 
    sorted_train['sequence'][len(sorted_train)-2]
)

print(pairwise2.format_alignment(*alignment[0]))

sorted_train.tail(5)

GSSET-RYRFTDEEE-ARRAAKEWARRGYQVHVTQNGTYWEVEVR
||| | |||||| || |||||||||||||||||||||||||||||
GSS-TTRYRFTD-EEKARRAAKEWARRGYQVHVTQNGTYWEVEVR
  Score=41



,index,sequence,consensus_stability_score
7705,5218,ERRKIEEIAKKLYQSGNPEAARWFLRKAGISEEEIERILQKAG,2.13
7706,7132,ERRKIEEIAKKLYQSGNPEAARKFLRKAGISEEEIERILQKAG,2.20
7707,1541,GSSKTQYEYDTKEEAQKAYEKFKRQGIPVTITQKNGKWFVQVE,2.28
7708,575,GSSTTRYRFTDEEKARRAAKEWARRGYQVHVTQNGTYWEVEVR,2.47
7709,3655,GSSETRYRFTDEEEARRAAKEWARRGYQVHVTQNGTYWEVEVR,2.49


## Subsampling techniques

* Diversity Maximizing: This is a greedy strategy which starts from the reference and adds the sequence with highest average hamming distance to current set of sequences.

* Diversity Minimizing: This strategy is equivalent to the Diversity Maximizing strategy, but adds the se- quence with lowest average hamming distance. It is used to explore the effects of diversity on model per- formance.

* HHFilter: This strategy applies hhfilter (Steinegger etal.,2019)withthe-diff Mparameter,whichre- turns M or more sequences that maximize diversity (the result is usually close to M). If more than M sequences are returned we apply the Diversity Maxi- mizing strategy on top of the output.

In [299]:
import jax.numpy as jnp
import numpy as np

def onehot_pair(u, v):
    vocab = set(u).union(set(v))
    mapping = {k: v for v, k in enumerate(vocab)}
    
    for idx, val in enumerate(u):
        u[idx] = mapping[val]
    
    for idx, val in enumerate(v):
        v[idx] = mapping[val]
        
    return u, v

def onehot_arrays(arrays):
    vocab = set()
    for array in arrays[1:]:
        vocab = vocab.union(set([char for char in array]))
    
    mapping = {k: v for v, k in enumerate(vocab)}
    
    array_final = np.empty_like(arrays)
    for idx, string in enumerate(arrays):
        array = np.array([char for char in string])
        for arr_idx, val in enumerate(array):
            array[arr_idx] = mapping[val]
        array_final[idx] = array
        
    return array_final

def _validate_vector(u, dtype=None):
    u = jnp.asarray(u, dtype=dtype)
    if u.ndim == 1:
        return u

    # Ensure values such as u=1 and u=[1] still return 1-D arrays.
    u = jnp.atleast_1d(u.squeeze())
    if u.ndim > 1:
        raise ValueError("Input vector should be 1-D.")
    warnings.warn(
        "scipy.spatial.distance metrics ignoring length-1 dimensions is "
        "deprecated in SciPy 1.7 and will raise an error in SciPy 1.9.",
        DeprecationWarning)
    return u

def hamming(u, v):
    if np.array(u).dtype == "<U1" and np.array(v).dtype == "<U1":
        u, v = onehot_pair(u, v)
    u = _validate_vector(u)
    v = _validate_vector(v)
    
    u_ne_v = u != v
    return np.average(u_ne_v)

In [300]:
import random

from numba import jit

from tqdm import tqdm
import jax

def compute_diversity(seq1, seq2, method="hamming"):
    if method == "alignment":
        method = pairwise2.align.globalxx
        diversity = method(seq1, seq2)
        diversity = diversity[0].score / max(len(seq1), len(seq2))

    elif method == "hamming":
        if len(seq1) != len(seq2):
            return np.nan

        method = hamming
        diversity = method(list(seq1), list(seq2))

    return diversity

def dataset_diversity(sequences, method="hamming", reduce="mean", verbose=True):
    if reduce != "mean":
        raise NotImplementedError
    else:
        # nan results are due to different lengths
        reducer = np.nanmean

    reduced_divs = []
    if verbose:
        pbar = tqdm(total=len(sequences), miniters=1, smoothing=1)

    all_other_sequences = [
        np.concatenate((sequences[:idx], sequences[idx + 1 :]))
        for idx in range(len(sequences))
    ]
    
    for seq_idx, seq in enumerate(sequences):
        other_sequences = onehot_arrays(all_other_sequences[seq_idx])
        print(other_sequences)
        if seq in other_sequences:
            raise IndexError(
                f"{seq_idx} is in other_sequences, e.g. it is not correctly indexed"
            )

        v_diversity = jax.vmap(lambda x: compute_diversity(seq, x))
        if len(other_sequences) > 1:
            
            div_vs_all = v_diversity()
        else:
            print(f"Skipping sequence {seq_idx}")
            continue

        reduced_div_vs_all = reducer(div_vs_all) if len(div_vs_all) >= 1 else np.nan
        reduced_divs.append(reduced_div_vs_all)

        if verbose:
            pbar.update(1)
            pbar.refresh()

    if verbose:
        pbar.close()

    return reduced_divs

In [301]:
compute_diversity(
    sets["train"]["sequence"].values[0],
    sets["train"]["sequence"].values[11],
    "alignment",
)

0.9534883720930233

In [302]:
compute_diversity(sets["train"]["sequence"].values[0], sets["train"]["sequence"].values[11])

0.046511627906976744

In [303]:
diversities_train = dataset_diversity(sets["train"]["sequence"].values)



  0%|          | 0/7710 [00:00<?, ?it/s]

[array(['5', '9', '1', '8', '1', '1', '8', '9', '1', '8', '1', '1', '7',
       '7', '8', '1', '5', '1', '0', '1', '1', '2', '1', '1', '1', '1',
       '8', '1', '6', '1', '8', '1', '8', '1', '6', '1', '1', '8', '1',
       '8', '7', '2', '7'], dtype='<U1')
 array(['5', '8', '9', '7', '7', '7', '9', '8', '8', '1', '9', '7', '7',
       '1', '8', '8', '1', '1', '1', '7', '1', '2', '1', '4', '8', '4',
       '1', '2', '5', '3', '8', '1', '1', '1', '1', '7', '1', '1', '8',
       '9', '9', '8', '7'], dtype='<U1')
 array(['1', '3', '3', '1', '8', '5', '4', '8', '1', '8', '1', '8', '8',
       '8', '1', '1', '1', '1', '1', '7', '8', '9', '1', '7', '5', '1',
       '1', '8', '1', '7', '4', '8', '1', '1', '1', '2', '7', '6', '1',
       '1', '1', '1', '5'], dtype='<U1')
 ...
 array(['5', '1', '1', '8', '4', '4', '7', '1', '9', '8', '1', '1', '1',
       '7', '1', '2', '7', '1', '4', '1', '1', '1', '2', '1', '5', '1',
       '5', '3', '1', '1', '8', '1', '8', '7', '9', '1', '7', '7', '9',
    

/home/roberto/anaconda3/envs/protera-stability/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


ValueError: vmap must have at least one non-None value in in_axes

In [83]:
diversities_train[:10]

[0.8394134005304014,
 0.8469666259574454,
 0.8795092981365058,
 0.83650682658194,
 0.8295142928034904,
 0.8158611289236332,
 0.836996626183011,
 0.043124059151686885,
 0.8282833490692195,
 0.87912583660672]

In [84]:
pd.read_csv("data/stability_diversity_train.csv", index_col=0)[:10]

,sequence,consensus_stability_score,diversity
0,GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT,0.37,0.839413
1,TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK,0.62,0.846967
2,TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK,-0.03,0.879509
3,GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT,1.41,0.836507
4,TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT,1.11,0.829514
5,SKDEAQREAERAIRSGNKEEARRILEEVGYSPEQAERIIRKLG,1.24,0.815861
6,TIDEIIKALEQAVKDGKPIQVGNYTVTSADEAEKLAKKLKKEY,1.05,0.836997
7,FEIPDDVPLPAGWEMARTSSGQRYFKNHIDQTTTWQDPRKAMLSQM,0.89,0.043124
8,TTIHVGDLTLKYDNPKKAYEIAKKLDKKYNLTVTIKNGKITVT,0.88,0.828283
9,GSSGSLSDEDFKAVFGMTRSAFAMLPLWKQQNLKKEKGLFGSS,1.15,0.879126


In [48]:
# ds_div_train = dataset_diversity(sets["train"]["sequence"])
# sets["train"]["diversity"] = np.nanmean(np.array(ds_div_train), 1)
# np.nanmean(np.array(ds_div_train))
# 0.8xxx

In [49]:
# ds_div_test = dataset_diversity(sets["test"]["sequence"])
# sets["test"]["diversity"] = np.nanmean(np.array(ds_div_test), 1)
# np.nanmean(np.array(ds_div_test))

In [50]:
# sets["train"].to_csv(f"{data_path}/stability_diversity_train.csv")
# sets["test"].to_csv(f"{data_path}/stability_diversity_test.csv")

In [51]:
sets = {
    'train' : pd.read_csv(f"{data_path}/stability_diversity_train.csv"),
    'test'  : pd.read_csv(f"{data_path}/stability_diversity_test.csv") 
}

sorted_div = {
    "train" : sets["train"].sort_values(by="diversity", axis=0, ascending=False),
    "test"  : sets["test"].sort_values(by="diversity", axis=0, ascending=False),
}

In [226]:
import torch
from torch.utils.data import Dataset

class ProteinStabilityDataset(Dataset):
    """Protein1D Stability dataset."""

    def __init__(self, proteins_path, ret_dict = True):
        """
        Args:
            proteins_path (string): Path to the H5Py file that contains sequences and embeddings.
        """
        self.stability_path = proteins_path
        self.ret_dict = ret_dict

    def __len__(self):
        return len(h5py.File(self.stability_path, "r")['sequences'])
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        with h5py.File(self.stability_path, "r") as dset:
            sequences = dset['sequences'][idx]
            embeddings = dset['embeddings'][idx]
            labels = dset['labels'][idx]
            
            sample = {
                'sequences': sequences, 
                'embeddings': torch.from_numpy(embeddings), 
                'labels': torch.Tensor([labels])
            }

        return sample if self.ret_dict else (sample['embeddings'], sample['labels'])

In [227]:
from torch.utils.data import Sampler, DataLoader, SubsetRandomSampler
from typing import List, Sized, Iterator

import numpy as np

class SubsetDiversitySampler(Sampler):
    """Samples elements given their diversity w.r.t. the rest of the dataset"""
    
    def __init__(self, dataset: Sized, diversity_path : str, diversity_cutoff : float, strategy : str = "maximize", seed : int = 123) -> None:
        """
        Args:
          diversity_path (string): Path to the csv with sequences and diversity.
        """
        self.diversity_path = diversity_path
        self.diversity_data = pd.read_csv(self.diversity_path, index_col=0)
        
        if strategy == "maximize":
            sorting_order = False
        elif strategy == "minimize":
            sorting_order = True
        else: 
            raise ValueError(f"Strategy {strategy} is not supported")
        
        self.diversity_data = self.diversity_data.sort_values(
            by='diversity', 
            ascending=sorting_order
        )
        
        self.cutoff = diversity_cutoff
        self.indices = []
        self.strategy = strategy
        self.seed = seed
        
        if strategy == 'maximize':
            self.cutoff_lambda = lambda x, cutoff : x < cutoff
        elif strategy == 'minimize':
            self.cutoff_lambda = lambda x, cutoff : x > cutoff
            
        self.subset_by_cutoff(self.cutoff, self.cutoff_lambda)
            
    def subset_by_cutoff(self, cutoff, cutoff_func) -> None:
        for row in self.diversity_data.itertuples():
            is_cutoff = cutoff_func(row.diversity, cutoff)
            if is_cutoff:
                break
            else:
                self.indices.append(row.Index)
        
    def __iter__(self) -> Iterator[int]:
        rng = np.random.default_rng(self.seed)
        return iter(rng.permutation(self.indices))

    def __len__(self) -> int:
        return len(self.indices)

In [238]:
dataset = ProteinStabilityDataset("data/stability.h5", ret_dict=True)
sampler = SubsetDiversitySampler(dataset, "data/stability_diversity_full.csv", 0.85)
dl = DataLoader(dataset, batch_size=128, sampler=sampler)

# check diversity sampler
df = pd.read_csv("data/stability_diversity_full.csv", index_col=0)
for batch in dl:
    for seq in batch["sequences"]:
        row = df[df.sequence == str(seq, 'utf-8')]
        if float(row.diversity.values) < 0.85:
            raise AssertionError

In [218]:
# for s in [1000, 1500, 2000, 3000, 4000, 5000, 6000]:
#     for kind in ["train"]:
#         chosen_by_diversity = select_by_diversity(
#             sequences=sorted_div[kind]["sequence"].values, 
#             diversities=sorted_div[kind]["diversity"].values,
#             min_size=s, 
#             min_score=0.78
#         )
    
#     subset = sets["train"][sets["train"]["sequence"].isin(chosen_by_diversity)]
#     subset.to_csv(f"{data_path}/stability_diversity_train_{s}.csv", index=False)